In [11]:
pip install keras


Note: you may need to restart the kernel to use updated packages.


In [14]:
pip install keras.engine.topology

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement keras.engine.topology (from versions: none)
ERROR: No matching distribution found for keras.engine.topology


In [8]:

import numpy as np # linear algebra
import pandas as pd 



import os
print(os.listdir("input"))

# Any results you write to the current directory are saved as output.

['voice.csv']


In [15]:
import matplotlib.pylab as plt
from scipy import interp
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_curve,auc
from sklearn.model_selection import StratifiedKFold
import matplotlib.patches as patches
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output


from keras import Sequential
from keras.utils import pad_sequences
from sklearn.model_selection import KFold
from sklearn.metrics import *
from keras.models import Sequential,Model
from keras.layers import *
from keras import backend as K
from tensorflow.keras.layers import Layer, InputSpec
from keras import initializers, regularizers, constraints

In [16]:
data = pd .read_csv('input/voice.csv')

In [17]:
data.head(10)

,meanfreq,sd,median,Q25,Q75,IQR,skew,kurt,sp.ent,sfm,...,centroid,meanfun,minfun,maxfun,meandom,mindom,maxdom,dfrange,modindx,label
0,0.059781,0.064241,0.032027,0.015071,0.090193,0.075122,12.863462,274.402906,0.893369,0.491918,...,0.059781,0.084279,0.015702,0.275862,0.007812,0.007812,0.007812,0.000000,0.000000,male
1,0.066009,0.067310,0.040229,0.019414,0.092666,0.073252,22.423285,634.613855,0.892193,0.513724,...,0.066009,0.107937,0.015826,0.250000,0.009014,0.007812,0.054688,0.046875,0.052632,male
2,0.077316,0.083829,0.036718,0.008701,0.131908,0.123207,30.757155,1024.927705,0.846389,0.478905,...,0.077316,0.098706,0.015656,0.271186,0.007990,0.007812,0.015625,0.007812,0.046512,male
3,0.151228,0.072111,0.158011,0.096582,0.207955,0.111374,1.232831,4.177296,0.963322,0.727232,...,0.151228,0.088965,0.017798,0.250000,0.201497,0.007812,0.562500,0.554688,0.247119,male
4,0.135120,0.079146,0.124656,0.078720,0.206045,0.127325,1.101174,4.333713,0.971955,0.783568,...,0.135120,0.106398,0.016931,0.266667,0.712812,0.007812,5.484375,5.476562,0.208274,male
5,0.132786,0.079557,0.119090,0.067958,0.209592,0.141634,1.932562,8.308895,0.963181,0.738307,...,0.132786,0.110132,0.017112,0.253968,0.298222,0.007812,2.726562,2.718750,0.125160,male
6,0.150762,0.074463,0.160106,0.092899,0.205718,0.112819,1.530643,5.987498,0.967573,0.762638,...,0.150762,0.105945,0.026230,0.266667,0.479620,0.007812,5.312500,5.304688,0.123992,male
7,0.160514,0.076767,0.144337,0.110532,0.231962,0.121430,1.397156,4.766611,0.959255,0.719858,...,0.160514,0.093052,0.017758,0.144144,0.301339,0.007812,0.539062,0.531250,0.283937,male
8,0.142239,0.078018,0.138587,0.088206,0.208587,0.120381,1.099746,4.070284,0.970723,0.770992,...,0.142239,0.096729,0.017957,0.250000,0.336476,0.007812,2.164062,2.156250,0.148272,male
9,0.134329,0.080350,0.121451,0.075580,0.201957,0.126377,1.190368,4.787310,0.975246,0.804505,...,0.134329,0.105881,0.019300,0.262295,0.340365,0.015625,4.695312,4.679688,0.089920,male


In [18]:
print(data.columns)

Index(['meanfreq', 'sd', 'median', 'Q25', 'Q75', 'IQR', 'skew', 'kurt',
       'sp.ent', 'sfm', 'mode', 'centroid', 'meanfun', 'minfun', 'maxfun',
       'meandom', 'mindom', 'maxdom', 'dfrange', 'modindx', 'label'],
      dtype='object')


In [19]:
label_value_count = data.label.value_counts()
print(label_value_count)
print(data.info())

male      1584
female    1584
Name: label, dtype: int64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3168 entries, 0 to 3167
Data columns (total 21 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   meanfreq  3168 non-null   float64
 1   sd        3168 non-null   float64
 2   median    3168 non-null   float64
 3   Q25       3168 non-null   float64
 4   Q75       3168 non-null   float64
 5   IQR       3168 non-null   float64
 6   skew      3168 non-null   float64
 7   kurt      3168 non-null   float64
 8   sp.ent    3168 non-null   float64
 9   sfm       3168 non-null   float64
 10  mode      3168 non-null   float64
 11  centroid  3168 non-null   float64
 12  meanfun   3168 non-null   float64
 13  minfun    3168 non-null   float64
 14  maxfun    3168 non-null   float64
 15  meandom   3168 non-null   float64
 16  mindom    3168 non-null   float64
 17  maxdom    3168 non-null   float64
 18  dfrange   3168 non-null   float64
 19  modindx   316

In [20]:
dict = {'label':{'male':1,'female':0}}      # label = column name
data.replace(dict,inplace = True)           # replace = str to numerical
x = data.loc[:, data.columns != 'label']
y = data.loc[:,'label']

In [21]:
x.head()

,meanfreq,sd,median,Q25,Q75,IQR,skew,kurt,sp.ent,sfm,mode,centroid,meanfun,minfun,maxfun,meandom,mindom,maxdom,dfrange,modindx
0,0.059781,0.064241,0.032027,0.015071,0.090193,0.075122,12.863462,274.402906,0.893369,0.491918,0.000000,0.059781,0.084279,0.015702,0.275862,0.007812,0.007812,0.007812,0.000000,0.000000
1,0.066009,0.067310,0.040229,0.019414,0.092666,0.073252,22.423285,634.613855,0.892193,0.513724,0.000000,0.066009,0.107937,0.015826,0.250000,0.009014,0.007812,0.054688,0.046875,0.052632
2,0.077316,0.083829,0.036718,0.008701,0.131908,0.123207,30.757155,1024.927705,0.846389,0.478905,0.000000,0.077316,0.098706,0.015656,0.271186,0.007990,0.007812,0.015625,0.007812,0.046512
3,0.151228,0.072111,0.158011,0.096582,0.207955,0.111374,1.232831,4.177296,0.963322,0.727232,0.083878,0.151228,0.088965,0.017798,0.250000,0.201497,0.007812,0.562500,0.554688,0.247119
4,0.135120,0.079146,0.124656,0.078720,0.206045,0.127325,1.101174,4.333713,0.971955,0.783568,0.104261,0.135120,0.106398,0.016931,0.266667,0.712812,0.007812,5.484375,5.476562,0.208274


In [22]:
y.head()

0    1
1    1
2    1
3    1
4    1
Name: label, dtype: int64

In [25]:
x = x.values
y = y.values

In [26]:
print(x)

[[0.05978098 0.06424127 0.03202691 ... 0.0078125  0.         0.        ]
 [0.06600874 0.06731003 0.04022873 ... 0.0546875  0.046875   0.05263158]
 [0.0773155  0.08382942 0.03671846 ... 0.015625   0.0078125  0.04651163]
 ...
 [0.14205626 0.09579843 0.18373124 ... 2.9375     2.9296875  0.19475862]
 [0.14365874 0.09062826 0.18497617 ... 3.59375    3.5859375  0.31100218]
 [0.16550895 0.09288354 0.18304392 ... 0.5546875  0.546875   0.35      ]]


In [27]:
print(y)

[1 1 1 ... 0 0 0]


In [28]:
from sklearn.utils import shuffle
x, y = shuffle(x, y, random_state=1010101)

In [29]:
trainX = x[:int(len(x) * 0.7)]
trainY = y[:int(len(y) * 0.7)]
validateX = x[int(len(x) * 0.7) : int(len(x) * 0.9)]
validateY = y[int(len(y) * 0.7) : int(len(y) * 0.9)]
testX = x[int(len(x) * 0.9):]
testY = y[int(len(y) * 0.9):]

print (len(trainX))
print (len(validateX))
print (len(testX))

2217
634
317


In [30]:
trainX = np.array(trainX)
trainY = np.array(trainY)
validateX = np.array(validateX)
validateY = np.array(validateY)
testX = np.array(testX)
testY = np.array(testY)

In [31]:
print (trainX.shape)

(2217, 20)


In [40]:
model = Sequential()
model.add(Dense(len(trainX[0]), input_dim=len(trainX[0]), activation='relu'))
model.add(Dense(100, activation='relu'))
model.add(Dense(100, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.save('gender_model.h5')

In [33]:
model.compile(loss='mse', optimizer='adam', metrics=['accuracy'])

In [35]:
history = model.fit(trainX, trainY, epochs=1000, batch_size=1000, validation_data = (validateX, validateY))

Epoch 1/1000
3/3 [==============================] - 0s 44ms/step - loss: 0.0834 - accuracy: 0.9062 - val_loss: 0.0995 - val_accuracy: 0.8785
Epoch 2/1000
3/3 [==============================] - 0s 19ms/step - loss: 0.0846 - accuracy: 0.9017 - val_loss: 0.0741 - val_accuracy: 0.9211
Epoch 3/1000
3/3 [==============================] - 0s 18ms/step - loss: 0.0746 - accuracy: 0.9157 - val_loss: 0.0770 - val_accuracy: 0.9101
Epoch 4/1000
3/3 [==============================] - 0s 23ms/step - loss: 0.0750 - accuracy: 0.9143 - val_loss: 0.0693 - val_accuracy: 0.9164
Epoch 5/1000
3/3 [==============================] - 0s 19ms/step - loss: 0.0724 - accuracy: 0.9188 - val_loss: 0.0729 - val_accuracy: 0.9164
Epoch 6/1000
3/3 [==============================] - 0s 23ms/step - loss: 0.0694 - accuracy: 0.9202 - val_loss: 0.0619 - val_accuracy: 0.9306
Epoch 7/1000
3/3 [==============================] - 0s 21ms/step - loss: 0.0656 - accuracy: 0.9292 - val_loss: 0.0616 - val_accuracy: 0.9290
Epoch 8/1000


In [36]:
predictions = model.predict(testX)

10/10 [==============================] - 0s 1ms/step


In [37]:
prob = []
limit_prob = []

for k in range(len(predictions)):
    prob.append(round(predictions[k][0], 4))
    if round(predictions[k][0], 4) > 0.5:
        limit_prob.append(1)
    else:
        limit_prob.append(0)

In [38]:
my_submission = pd.DataFrame({'real': testY, 'prediction': prob, 'limit prediction' : limit_prob})
# you could use any filename. We choose submission here
my_submission.to_csv('sample_submission1.csv', index=False)

In [39]:
my_submission.head(50)

,real,prediction,limit prediction
0,0,0.0005,0
1,1,0.9896,1
2,1,1.0000,1
3,1,0.9472,1
4,1,0.9915,1
5,1,0.9657,1
6,1,0.9811,1
7,0,0.0003,0
8,0,0.0000,0
9,1,0.9601,1
